In [1]:
import pandas as pd
import numpy as np
import statistics
import math 
from math import exp
from mpmath import mp

In [2]:
path = "/Users/harivigneshgomathi/Downloads/"
train_data = pd.read_csv(path+"1644871288_9762487_cleveland-train.csv")
test_data = pd.read_csv(path+"1644871288_9775174_cleveland-test.csv")

In [3]:
train_data_arr = train_data.values
test_data_arr = test_data.values

In [4]:
def meanandStandardDev(dataset):
    mean_std = []
    for i in range(len(dataset[0])):
        col_val = [j[i] for j in dataset]
        mean = np.average(col_val)
        std = np.std(col_val)
        mean_std.append([mean, std])
    return mean_std

In [5]:
def normalize(dataset,meanstd):
    for i in range(len(dataset)):
        for j in range(len(dataset[0])):
            n = dataset[i][j] - meanstd[j][0]
            d = meanstd[j][1]
            dataset[i][j] = n/d
    return dataset

In [6]:
def predict(data,weight):
    z = weight[0]
    for i in range(len(data)-1):
        z += weight[i + 1] * data[i]
    return 1 / (1 + np.exp(-z))

In [7]:
def cross_ent_error(dataset,weight):
    error = 0
    for tuple in dataset:
        pred = predict(tuple,weight)
        label = tuple[-1]
        error += -(label*np.log(pred))+(-(1-label)*np.log(1-pred))
    cross_ent_err = error/len(dataset)
    return cross_ent_err

In [8]:
def gradient_descent(train_data, learning_rate, iteration):
    weight = [0.0 for i in range(len(train_data[0]))]
    ch_weight = [0.0 for i in range(len(train_data[0]))]
    crEnt_err = []
    for i in range(iteration):
        for tuple in train_data:
            pred = predict(tuple,weight)
            ch_weight[0] = ch_weight[0] + (tuple[-1] - pred)*pred*(1 - pred)
            for i in range(len(tuple)-1):
                ch_weight[i+1] = ch_weight[i+1] + ((tuple[-1] - pred)*pred*(1 - pred)*tuple[i])
        for i in range(len(tuple)-1):
            weight[i] = weight[i] - (learning_rate*-2*ch_weight[i])
        crEnt_err.append(cross_ent_error(train_data,weight))
        
    return crEnt_err,weight 

In [9]:
def logistic_regression(train_data,test_data,learning_rate,iteration):
    pred = list()
    cross_ent_error,weight = gradient_descent(train_data,learning_rate,iteration)
    for tuple in test_data:
        z=predict(tuple,weight)
        pred.append(z)
    return pred,weight,cross_ent_error

In [10]:
def classification_error(train_data,weight):
    pred_Cl_err = list()
    for tuple in train_data:
        z=predict(tuple,weight)
        pred_Cl_err.append(z)
    pred_Cl_err = [1 if i >0.5 else -1 for i in pred_Cl_err]
    count = 0
    return pred_Cl_err

In [11]:
train_minmax = meanandStandardDev(train_data_arr)
test_minmax = meanandStandardDev(test_data_arr)
train_data_arr = normalize(train_data_arr,train_minmax)
test_data_arr = normalize(test_data_arr,test_minmax)

In [12]:
eta = 10**-5

In [13]:
pred,weight,ce_errors = logistic_regression(train_data_arr,test_data_arr,eta,10000)

/var/folders/4_/2p34smp57y1gjj525ytht7dr0000gn/T/ipykernel_6096/1012866973.py:6: RuntimeWarning: divide by zero encountered in log
  error += -(label*np.log(pred))+(-(1-label)*np.log(1-pred))
/var/folders/4_/2p34smp57y1gjj525ytht7dr0000gn/T/ipykernel_6096/1012866973.py:6: RuntimeWarning: invalid value encountered in double_scalars
  error += -(label*np.log(pred))+(-(1-label)*np.log(1-pred))
/var/folders/4_/2p34smp57y1gjj525ytht7dr0000gn/T/ipykernel_6096/62958044.py:5: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


In [14]:
pred = [1 if i >0.5 else -1 for i in pred]

In [15]:
#Testing for classification error using train data
train_pred = classification_error(train_data_arr,weight)

/var/folders/4_/2p34smp57y1gjj525ytht7dr0000gn/T/ipykernel_6096/62958044.py:5: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


In [16]:
# Calculating the classification accuracy
count = 0
for i in range(len(train_pred)):
    if(train_pred[i] == train_data.iloc[i,-1]):
        count+=1
accuracy = count/len(train_pred)
classification_error = 1 - accuracy

In [17]:
classification_error

0.16447368421052633

In [18]:
pred_dataFrame = pd.DataFrame(pred)
pred_dataFrame.to_csv(path+"Log_Reg_OP_predictions_v2.dat", index=False, header=None)

In [19]:
# Prediction using Scikit library.
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
lr_model.fit(train_data.iloc[:,0:13].values,train_data.iloc[:,-1])
y_pred_sk = lr_model.predict(test_data_arr)
y_pred_sk_train = lr_model.predict(train_data.iloc[:,0:13].values)

/Users/harivigneshgomathi/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
pred_dataFrame = pd.DataFrame(y_pred_sk)
pred_dataFrame.to_csv(path+"Log_Reg_OP_predictions_lib.dat", index=False, header=None)

In [21]:
c = 0
for i in range(len(y_pred_sk)):
    if(y_pred_sk[i] == train_data.iloc[i,-1]):
        c+=1

In [22]:
d = c/len(y_pred_sk)

In [23]:
#Classification on train dataset using logistic regression output
1-d

0.4896551724137931